<a href="https://colab.research.google.com/github/Haebuk/AutoML/blob/main/logfile_practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm

for path, dir, filenames in os.walk('/content/drive/MyDrive/input/access.log/'):
    for filename in filenames:
        logfile = os.path.join(path, filename)
        print(logfile)

/content/drive/MyDrive/input/access.log/access.log


In [ ]:
with open(logfile) as log:
    line_num = 0
    for i in log:
        print(i)
        line_num += 1
        if line_num == 5: break

54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"

31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"

31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"

40.77.167.129 - - [22/Jan/2019:03:56:17 +0330] "GET /image/14925/productModel/100x100 HTTP/1.1" 200 16

In [ ]:
regex = '^(?P<IP>\S+) - - \[(?P<datetime>[^*]+)\] "(?P<method>\S+) (?P<request>\S+) HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+) "(?P<url>\S+) "(?P<agent>[^\^]+)"'
columns = ['IP', 'datetime', 'method', 'request', 'status', 'size', 'url', 'agent']

In [ ]:
%mkdir parquet_dir

In [ ]:
def logfile_processing(logfile, output_dir, error_file):
    with open(logfile) as log:
        line_num = 0
        parsed_list = []
        for line in tqdm(log):
            
            try:
                parsed_line = re.findall(regex, line)[0]
                if line_num % 100000 == 0:
                    print(line)
                parsed_list.append(parsed_line)
            except Exception as e:
                with open(error_file, 'at') as error:
                    print((line, str(e)), file=error)
                continue
            line_num += 1
            if line_num % 250000 == 0:
                df = pd.DataFrame(parsed_list, columns=columns)
                df.to_parquet(f'{output_dir}/file_{line_num}.parquet')
                parsed_list.clear()

        df = pd.DataFrame(parsed_list, columns=columns)
        df.to_parquet(f'{output_dir}/file_{line_num}.parquet')
        parsed_list.clear()

In [ ]:
%time logfile_processing(logfile=logfile, output_dir='parquet_dir', error_file='errors.txt')

21133it [00:00, 106052.99it/s]

54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"



116714it [00:01, 106893.35it/s]

95.81.72.148 - - [22/Jan/2019:08:01:41 +0330] "GET /image/53306/productModel/150x150 HTTP/1.1" 200 3499 "https://www.zanbil.ir/browse/digital-supplies/%D9%84%D9%88%D8%A7%D8%B2%D9%85-%D8%AF%DB%8C%D8%AC%DB%8C%D8%AA%D8%A7%D9%84" "Mozilla/5.0 (Windows NT 6.1; rv:60.0) Gecko/20100101 Firefox/60.0" "-"



212292it [00:02, 103696.35it/s]

...

2.191.144.134 - - [26/Jan/2019:19:18:41 +0330] "GET /image/399/brand HTTP/1.1" 200 2302 "https://www.zanbil.ir/browse/hood/%D9%87%D9%88%D8%AF" "Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20100101 Firefox/64.0" "-"



10318805it [02:28, 71407.73it/s]

86.55.88.52 - - [26/Jan/2019:19:59:04 +0330] "GET /favicon.ico HTTP/1.1" 200 0 "https://www.zanbil.ir/m" "Mozilla/5.0 (Linux; Android 7.0; TRT-L21A) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.80 Mobile Safari/537.36" "-"



10365152it [02:29, 69453.96it/s]


CPU times: user 2min 17s, sys: 9.58 s, total: 2min 26s
Wall time: 2min 30s


In [ ]:
!wc errors.txt

   294   7223 230187 errors.txt


In [ ]:
%time logs_df = pd.read_parquet('parquet_dir/')

CPU times: user 15.5 s, sys: 4.57 s, total: 20.1 s
Wall time: 14.8 s


In [ ]:
!du -sh parquet_dir/

257M	parquet_dir/


In [ ]:
logs_df.head()

,IP,datetime,method,request,status,size,url,agent
0,37.152.163.59,22/Jan/2019:12:38:27 +0330,GET,/static/images/loading.gif,200,7370,https://www.zanbil.ir/product/29314/%DA%A9%D8%...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
1,77.245.233.52,22/Jan/2019:12:38:27 +0330,GET,/image/11082/productType/240x180,200,12458,https://www.zanbil.ir/browse/sports/%D8%AA%D8%...,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...
2,37.27.128.139,22/Jan/2019:12:38:27 +0330,GET,/browse/Tablet-Arm-Chair/%D8%B5%D9%86%D8%AF%D9...,200,30604,https://www.zanbil.ir/browse/Classroom-Furnitu...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...
3,77.245.233.52,22/Jan/2019:12:38:27 +0330,GET,/image/851/mainSlide,200,89859,https://www.zanbil.ir/browse/sports/%D8%AA%D8%...,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...
4,77.245.233.52,22/Jan/2019:12:38:27 +0330,GET,/image/848/mainSlide,200,93168,https://www.zanbil.ir/browse/sports/%D8%AA%D8%...,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...


In [ ]:
logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364858 entries, 0 to 10364857
Data columns (total 8 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   IP        object
 1   datetime  object
 2   method    object
 3   request   object
 4   status    object
 5   size      object
 6   url       object
 7   agent     object
dtypes: object(8)
memory usage: 632.6+ MB
